Steps performed in this notebook:

**1. Load in spectral data:** The data from GNPS 24-04-09 all_positive is loaded in. It contains 187,152 spectra that are annotated with structures. They are filtered on unique inchikey, which 

**2. Create embeddings:** 
[Spec2Vec](https://github.com/iomega/spec2vec) and [MS2DeepScore](https://github.com/matchms/ms2deepscore) learn relationships between peaks as they create spectral embeddings. Within these embeddings, similar spectra are placed nearby each other in the latent space. This is also true for molecular embeddings, for example ones made with [Mol2Vec](https://github.com/samoturk/mol2vec).

Data is saved to a file to be reused by other notebooks.

In [22]:
!pwd
!python --version

/lustre/BIF/nobackup/unen004/thesis/cca
Python 3.8.12


## 1. Load in spectral data
GNPS 21-04-09 all positive

In [23]:
import os 
import pickle
import pandas as pd

path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings"
dataset = "ALL_GNPS_210409_positive"

# load spectra annotated with molecular structures
spectra_fn = os.path.join(path, "datasets", dataset,
                          "ALL_GNPS_210409_positive_cleaned_peaks_processed_s2v_only_annotated.pickle")
with open(spectra_fn, "rb") as f:
    all_spectrums = pickle.load(f)
    
print(len(all_spectrums))

187152


In [24]:
# Load in class info to merge into df later 
class_fn = os.path.join(path, "classifications", dataset,
                       "ALL_GNPS_210409_positive_cleaned_peaks_processed_s2v_only_annotated_classes.txt")
classes = pd.read_table(class_fn, sep = "\t") #, on_bad_lines="skip"

### Filter on unique inchikey
Function obtained from [Joris Louwen](https://github.com/louwenjjr/ms2_mass_differences/blob/ffd31aff66fba14502e3c7ec4f1b5eb947687ef1/scripts/mass_differences/processing.py).

In [25]:
from collections import OrderedDict
import numpy as np

def count_higher_peaks(spectrum, threshold = 0.1):
    return np.sum(spectrum.peaks.intensities/spectrum.peaks.intensities.max() >= threshold)

def get_ids_for_unique_inchikeys(spectrums): #: List[SpectrumType]
    """Return indices for best chosen spectra for each unique inchikey
    Parameters
    ----------
    spectrums:
        Input spectra
    """
    # collect all inchikeys (first 14 characters)
    inchikey_collection = OrderedDict()
    for i, spec in enumerate(spectrums):
        inchikey = spec.get("inchikey")
        if inchikey:
            if inchikey[:14] in inchikey_collection:
                inchikey_collection[inchikey[:14]] += [i]
            else:
                inchikey_collection[inchikey[:14]] = [i]

    intensity_thres = 0.01
    n_peaks_required = 10
    ID_picks = []

    inchikey14_unique = [x for x in inchikey_collection.keys()]

    # Loop through all unique inchiques (14 first characters)
    for inchikey14 in inchikey14_unique:
        specIDs = np.array(inchikey_collection[inchikey14])
        if specIDs.size == 1:
            ID_picks.append(specIDs[0])
        else:
            # 1 select spec with sufficient peaks (e.g. 10 with intensity 0.01)
            num_peaks = np.array([count_higher_peaks(
                spectrums[specID], intensity_thres) for
                                  specID in specIDs])
            sufficient_peaks = np.where(num_peaks >= n_peaks_required)[0]
            if sufficient_peaks.size == 0:
                sufficient_peaks = np.where(num_peaks == max(num_peaks))[0]
            step1IDs = specIDs[sufficient_peaks]

            # 2 select best spectrum qualities
            # (according to gnps measure). 1 > 2 > 3
            qualities = np.array(
                [int(spectrums[specID].get("library_class", 3))
                 for specID in step1IDs])  # default worst quality
            step2IDs = step1IDs[np.where(qualities == min(qualities))[0]]

            # 3 Select the ones with most peaks > threshold
            num_peaks = np.array([count_higher_peaks(
                spectrums[specID], intensity_thres) for specID in step2IDs])
            pick = np.argmax(num_peaks)
            ID_picks.append(step2IDs[pick])
    ID_picks.sort()  # ensure order

    return ID_picks

In [26]:
# filter on unique inchi keys
uniq_ids = get_ids_for_unique_inchikeys(all_spectrums)
uniq_spectrums = [all_spectrums[i] for i in uniq_ids]

# take small subset for dev
#spectrums = uniq_spectrums[:10000]
spectrums = uniq_spectrums

In [27]:
# turn into pandas dataframe
df = pd.DataFrame([spec.metadata for spec in spectrums])
df = df[['spectrum_id', 'compound_name', 'smiles', 'inchikey']]
df['inchikey14'] = [x[:14] for x in df['inchikey']]  # for tanimoto similarity matrix

# merge classes into main df
df = pd.merge(df, classes, on="spectrum_id")
print(len(df))

16360


## 1. Create embeddings


### MS2DeepScore

In [28]:
from ms2deepscore.models import load_model
from ms2deepscore import MS2DeepScore
from tqdm.notebook import tqdm  # optional, just to get a progress bar

data_path = '/lustre/BIF/nobackup/unen004/data'

def create_ms2ds_embeddings():
    """Create and save MS2DeepScore embeddings to file.
    """
    # Load pretrained ms2ds model
    ms2ds_model_fn = os.path.join(data_path, 'ms2ds_model_20210419-221701_data210409_10k_500_500_200.hdf5')
    ms2ds_model = load_model(ms2ds_model_fn)

    # Init MS2DeepScore
    ms2ds_score = MS2DeepScore(ms2ds_model)

    # Generate embeddings from spectra
    ms2ds_embeddings = ms2ds_score.calculate_vectors(spectrums)
    
    # Save embeddings to file, cause takes ~15 min to make the embeddings
    with open(os.path.join(data_path, 'ms2ds', 'ms2ds_embeddings.pickle'), 'wb') as f:
        pickle.dump(ms2ds_embeddings, f) 

def load_ms2ds_embeddings():    
    """Return ms2ds embedding vectors
    """
    with open(os.path.join(data_path, 'ms2ds', 'ms2ds_embeddings.pickle'), 'rb') as f:
        ms2ds_embeddings = pickle.load(f)
        return ms2ds_embeddings

#create_ms2ds_embeddings()
ms2ds_embeddings = load_ms2ds_embeddings()


### Spec2Vec

In [29]:
from gensim.models import Word2Vec
from spec2vec import SpectrumDocument
from spec2vec.vector_operations import calc_vector

def create_spec2vec_embeddings():
    """Create Spec2Vec embeddings 
    """
    # Load pretrained spec2vec model 
    spec2vec_model_fn = os.path.join(path, "embeddings", "ALL_GNPS_210409_positive", 
                            "ALL_GNPS_210409_positive_cleaned_spec2vec_embedding_iter_15.model")
    spec2vec_model = Word2Vec.load(spec2vec_model_fn)
    
    # Create "documents"
    spectrum_documents = [SpectrumDocument(s, n_decimals=2) for s in spectrums]
    
    # Derive embeddings from model with documents
    intensity_weighting_power = 0.5
    allowed_missing_percentage = 10 # specify the maximum (weighted) fraction of the spectrum that is allowed to be missing

    spec2vec_embeddings = np.zeros((len(spectrum_documents), spec2vec_model.vector_size), dtype="float")
    for i, doc in enumerate(tqdm(spectrum_documents)):
        spec2vec_embeddings[i, 0:spec2vec_model.vector_size] = calc_vector(spec2vec_model, doc,
                                                                           intensity_weighting_power,
                                                                           allowed_missing_percentage)
    
    return spec2vec_embeddings

# Add to dataframe
spec2vec_embeddings = create_spec2vec_embeddings()

  0%|          | 0/16360 [00:00<?, ?it/s]

### Mol2Vec

In [30]:
from mol2vec.features import mol2alt_sentence, MolSentence #,sentences2vec
from rdkit import Chem

def sentences2vec_new(sentences, model, unseen=None):
    """mol2vec.sentences2vec - but updated to Gensim version 4.
    https://github.com/samoturk/mol2vec/pull/16
    """
    keys = set(model.wv.key_to_index)
    vec = []
    
    if unseen:
        unseen_vec = model.wv.get_vector(unseen)

    for sentence in sentences:
        if unseen:
            vec.append(sum([model.wv.get_vector(y) if y in set(sentence) & keys
                       else unseen_vec for y in sentence]))
        else:
            vec.append(sum([model.wv.get_vector(y) for y in sentence 
                            if y in set(sentence) & keys]))
    return np.array(vec)

def create_mol2vec_embeddings():
    """Create Mol2Vec embeddings vectors
    """
    # Load pretrained mol2vec model 
    mol2vec_model_fn = os.path.join(data_path, 'mol2vec', 'model_300dim.pkl')
    mol2vec_model = Word2Vec.load(mol2vec_model_fn)

    df['mol'] = ""
    df['sentence'] = ""
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        mol = Chem.MolFromSmiles(row['smiles'])  # This gives the hydrogen warning
        if mol.GetNumAtoms() != 0:  
            df.at[i, 'mol'] = mol  # rdkit/molecule
            df.at[i, 'sentence'] = MolSentence(mol2alt_sentence(mol, radius=1))
    mol2vec_embeddings = [x for x in sentences2vec_new(df['sentence'], mol2vec_model, unseen='UNK')]
    return mol2vec_embeddings

mol2vec_embeddings = create_mol2vec_embeddings()

  0%|          | 0/16360 [00:00<?, ?it/s]

RDKit WARNING: [18:10:15] WARNING: not removing hydrogen atom without neighbors
[18:10:15] WARNING: not removing hydrogen atom without neighbors


## TODO: plot of normalization and non-normalization

In [31]:
# Normalize embedding vectors
from sklearn.preprocessing import normalize

# Normalize embeddings on samples
ms2ds_embeddings = normalize(ms2ds_embeddings)
spec2vec_embeddings = normalize(spec2vec_embeddings)
mol2vec_embeddings = normalize(mol2vec_embeddings)

In [32]:
# Add embedding vectors to df
df['ms2ds'] = [x for x in ms2ds_embeddings]
df['spec2vec'] = [x for x in spec2vec_embeddings]
df['mol2vec'] = [x for x in mol2vec_embeddings]

In [33]:
df.head()

,spectrum_id,compound_name,smiles,inchikey,inchikey14,cf_kingdom,cf_superclass,cf_class,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,npc_isglycoside,mol,sentence,ms2ds,spec2vec,mol2vec
0,CCMSLIB00000001547,3-Des-Microcystein_LR,CC(C)CC1NC(=O)C(C)NC(=O)C(=C)N(C)C(=O)CCC(NC(=...,IYDKWWDUBYWQGF-NNAZGLEUSA-N,IYDKWWDUBYWQGF,Organic compounds,Organic acids and derivatives,Peptidomimetics,Hybrid peptides,Hybrid peptides,Cyclic peptides; Microcystins,Oligopeptides,Amino acids and Peptides,0,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3537119515, 2245273601, 242354360...","[0.0, 0.11211989062426796, 0.0, 0.158931198460...","[0.03837159812841756, 0.09913249716376928, 0.0...","[0.005604683527400814, -0.02301042087378335, -..."
1,CCMSLIB00000001548,Hoiamide B,CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](...,KNGPFNUOXXLKCN-ZNCJFREWSA-N,KNGPFNUOXXLKCN,Organic compounds,Organic acids and derivatives,Peptidomimetics,Depsipeptides,Cyclic depsipeptides,Cyclic peptides,Oligopeptides,Amino acids and Peptides,0,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3542456614, 2245384272, 117312591...","[0.0, 0.10557166694366461, 0.0, 0.188627002322...","[0.07626190933359983, 0.1290448493814702, 0.06...","[-0.009834483545748103, -0.027489677816243393,..."
2,CCMSLIB00000001550,Scytonemin,OC1=CC=C(\C=C2\C(=O)C(C3=C4C5=C(C=CC=C5)N=C4\C...,CGZKSPLDUIRCIO-RPCRKUJJSA-N,CGZKSPLDUIRCIO,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,NaN,Indoles and derivatives,NaN,NaN,Shikimates and Phenylpropanoids,0,"<img data-content=""rdkit/molecule"" src=""data:i...","(864662311, 26234434, 3217380708, 2905660137, ...","[0.0, 0.1336975940932368, 0.0, 0.1912738648554...","[0.0792954864406949, 0.0829470276451531, -0.03...","[0.022136275297993648, -0.003164903719515904, ..."
3,CCMSLIB00000001552,Hectochlorin,C[C@H]1[C@@H](OC(C2=CSC([C@H](C(C)(OC(C3=CSC([...,USXIYWCPCGVOKF-LERJCCFDSA-N,USXIYWCPCGVOKF,Organic compounds,Organic acids and derivatives,Peptidomimetics,Depsipeptides,Cyclic depsipeptides,Cyclic peptides; Depsipeptides,Oligopeptides,Amino acids and Peptides; Polyketides,0,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 1858577693, 2976033787, 267264863...","[0.0, 0.0, 0.0, 0.20587242288185872, 0.2475291...","[0.005036696565324857, -0.057816207361243564, ...","[-0.0010993798911880348, -0.031671053774092295..."
4,CCMSLIB00000001554,Cyclomarin A,C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N([C@H](C(...,WCNJVJCYRBJSLC-BCJYPDSRSA-N,WCNJVJCYRBJSLC,Organic compounds,Phenylpropanoids and polyketides,Macrolactams,NaN,Macrolactams,Cyclic peptides,Oligopeptides,Amino acids and Peptides,0,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 1858577693, 2976033787, 125971071...","[0.06848110642982089, 0.0, 0.08852951477401143...","[-0.0067413807360351555, 0.03144355490931601, ...","[-0.0012273630584583826, -0.03138404125269303,..."


Lastly, save the dataframe to a file.

In [34]:
with open(os.path.join(data_path, 'full_dataframe.pickle'), 'wb') as f:
    pickle.dump(df, f)